In [ ]:
!pip install chromadb

In [ ]:
!pip -q install langchain openai tiktoken langchain-community

In [ ]:
import os
os.environ['OPENAI_API_KEY']="Place_your_api_key_here"

## Import some libraries

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from lanchain.document_loaders import DirectoryLoader

In [ ]:
loader=DirectoryLoader('./your_folder_path',glob="*.txt",loader_cls=TextLoader)
document=loader.load()

this will load the entire data inside the folder. So we need to devide it in different chunks. cause every llm model has some fixed token length which they can take input as.

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
texts=text_splitter.split_documents(document) # this will return an array of string with 1000 characters each

## Creating vector DB

In [ ]:
from lanchain import embeddings
persist_directory='db'
embeddings=openAIEmbeddings()
vectordb=Chroma.from_documents(documents=texts,embedding=embeddings,persist_directory=persist_directory)


vector have been created now we need to save it in disk

In [ ]:
# persist the db to disk
vectordb.persist()
vectordb=None

In [ ]:
# now we can load the persisted db from disk, and use it normally
vectordb=Chroma(persist_directory=persist_directory, embedding_function=embeddings)

## Make Retriever

In [ ]:
retriever=vectordb.as_retriever()
retriever=vectordb.as_retriever(search_type="similarity",search_kwargs={"k":2}) # this will help us to get how many response we want

In [ ]:
docs=retriever.get_relevant_documents("what is the meaning of life") # this code will return chunks of data from the documents based on the similarity search
# now we need to take help of llms to generate a valid response from the chunk of data and query we have provided.

## create chain

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
qa_chain=RetrievalQA.from_chain_type(llm=OpenAI(),chain_type="stuff",retriever=retriever,return_source_documents=True)


In [ ]:
query="what is the meaning of life"
result=qa_chain({"query":query})
result["result"]